In [2]:
# Vizuelna podešavanja okruženja (samo razvuče notebook na širinu ekrana)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


In [3]:
import networkx as nx
import pickle

with open('data_cleaned/data2018_cleaned', 'rb') as file:
    data = pickle.load(file)
print(data)

                          tourney_id                tourney_name surface  \
0                          2018-M020                    Brisbane    Hard   
1                          2018-M020                    Brisbane    Hard   
2                          2018-M020                    Brisbane    Hard   
3                          2018-M020                    Brisbane    Hard   
4                          2018-M020                    Brisbane    Hard   
...                              ...                         ...     ...   
2884                       2018-0605                 Tour Finals    Hard   
2885                       2018-0605                 Tour Finals    Hard   
2886  2018-M-DC-2018-WG-M-FRA-CRO-01  Davis Cup WG F: FRA vs CRO    Clay   
2887  2018-M-DC-2018-WG-M-FRA-CRO-01  Davis Cup WG F: FRA vs CRO    Clay   
2888  2018-M-DC-2018-WG-M-FRA-CRO-01  Davis Cup WG F: FRA vs CRO    Clay   

      draw_size tourney_level  tourney_date  match_num  winner_id  \
0            32   

In [4]:
lista_winners = set(data['winner_id'].unique())
lista_losers = set(data['loser_id'].unique())

svi_igraci = lista_winners.union(lista_losers)
print(f"Ukupan broj igraca je {len(svi_igraci)}, a ukupan broj meceva je {data.shape[0]}")


Ukupan broj igraca je 419, a ukupan broj meceva je 2889


In [5]:
import pandas as pd

dataPath = 'data/atp_players.csv'
players = pd.read_csv(dataPath)

dataPath = 'data/atp_rankings_10s.csv'
rankings = pd.read_csv(dataPath)

In [10]:
G = nx.Graph()
for player_id in svi_igraci:
    player = players[players['player_id'] == player_id]
    if player.empty:
        print("Postoji id igraca koji ne postoji u listi svih igraca!")
    else:
        player_rank = rankings[(rankings['player'] == player_id) & (rankings['ranking_date'] > 20180000) & (rankings['ranking_date'] < 20190000)]
        rank_p = 0
        points_p = 0
        
        if not player_rank.empty:
            last_row = player_rank.shape[0] - 1
            rank_p = int(player_rank.iloc[last_row, 1])
            points_p = int(player_rank.iloc[last_row, 3])
            
        G.add_node(str(player_id), first_name = str(player.iloc[0, 1]), last_name = str(player.iloc[0, 2]), country = str(player.iloc[0, 5]), hand = str(player.iloc[0, 3]), rank = rank_p, points = points_p)


In [11]:
data_reduced = data[['winner_id', 'loser_id']]
data_reduced = data_reduced.astype({'winner_id': str, 'loser_id': str})

for _, winner_id, loser_id in data_reduced.itertuples():
    if (winner_id, loser_id) in G.edges:
        G.edges[winner_id, loser_id]['weight'] += 1
    else:
        G.add_edge(winner_id, loser_id, weight = 1)

In [12]:
output_path = "models/model2018.gml"
nx.write_gml(G, output_path)

In [13]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 419
Number of edges: 2489
Average degree:  11.8807
